```
This software is part of GPU Ocean. 

Copyright (C) 2019  SINTEF Digital

Creating fan plots for plotting drift trajectory forecasts.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Plotting of drift trajectory forecasts


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from scipy.special import lambertw

import os
import sys
from importlib import reload

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common, IPythonMagic
from SWESimulators import CDKLM16, Common, DoubleJetCase, GPUDrifterCollection, Observation



In [ ]:
%cuda_context_handler gpu_ctx

# Create a simulator

... so that we have relevant parameters available

In [ ]:
%%time
doubleJetCase = DoubleJetCase.DoubleJetCase(gpu_ctx,
                                            DoubleJetCase.DoubleJetPerturbationType.IEWPFPaperCase)

doubleJetCase_args, doubleJetCase_init = doubleJetCase.getInitConditions()
sim = CDKLM16.CDKLM16(**doubleJetCase_args, **doubleJetCase_init)


# Define filenames and read drifter positions

In [ ]:
os.listdir('scripts/')
script_path = os.path.abspath('scripts/fanplotdemo')
truth_files = []
for subfolder in os.listdir('scripts/fanplotdemo/'):
    if subfolder.startswith('truth_2019_04_15-09'):
        drifter_file_name = os.path.join(os.path.join(script_path, subfolder),
                                         'drifter_observations.pickle')
        
        truth_files.append(drifter_file_name)
        
print(len(truth_files))

In [ ]:
observations = []
main_obs = Observation.Observation()
main_obs.read_pickle(truth_files[0])

first = True
for pickle in truth_files:
    if first:
        first = False
        continue
    obs = Observation.Observation()
    obs.read_pickle(pickle)
    
    observations.append(obs)
print(len(observations))

In [ ]:
obs_times = main_obs.get_observation_times()
print(obs_times)
print(len(obs_times))


In [ ]:
# Selecting one drifter and number of observation considered for plotting
drifter = 18 # 17
num_days = 3
observations_per_day = 24*12 # Every fifth minute
num_observations = int(num_days*observations_per_day)

extra_drifter = 51

In [ ]:
main_drifter_path = np.zeros((num_observations, 2))
extra_drifter_path = np.zeros((num_observations, 2))
for i in range(num_observations):
    obs_t = obs_times[i]
    main_drifter_path[i,:]  = main_obs.get_drifter_position(obs_t)[drifter,:]
    extra_drifter_path[i,:] = main_obs.get_drifter_position(obs_t)[extra_drifter,:]
#print(main_drifter_path)

In [ ]:
%%time
paths = [None]*len(observations)
extra_paths = [None]*len(observations)
for i in range(len(observations)):
    print('processing ensemble member ' + str(i))
    paths[i] = np.zeros((num_observations, 2))
    extra_paths[i] = np.zeros((num_observations, 2))
    for o in range(num_observations):
        obs_t = obs_times[o]
        paths[i][o,:] = observations[i].get_drifter_position(obs_t)[drifter,:]
        extra_paths[i][o,:] = observations[i].get_drifter_position(obs_t)[extra_drifter,:]


In [ ]:
%%time
extra_paths = []
for i in range(len(observations)):
    print('processing ensemble member ' + str(i))
    path = np.zeros((num_observations, 2))
    for o in range(num_observations):
        obs_t = obs_times[o]
        path[o,:] = observations[i].get_drifter_position(obs_t)[extra_drifter,:]
        if o > 0:
            e..?

# Create a canvas for plotting

In [ ]:

fig = plt.figure(figsize=(7,7))
ax = plt.subplot(111)
emptyData =np.ma.masked_where(np.zeros((sim.ny,sim.nx)) > 1, 
                              np.zeros((sim.ny,sim.nx)))
ax.imshow(emptyData, origin="lower", 
          extent=[0, sim.nx*sim.dx, 0, sim.ny*sim.dy], cmap='binary')


cell_id_x = int(np.floor(main_drifter_path[0,0]))
cell_id_y = int(np.floor(main_drifter_path[0,1]))


circ_start = matplotlib.patches.Circle((main_drifter_path[0,0], main_drifter_path[0,1]), 6000, fill=False)
ax.add_patch(circ_start)

circ_end = matplotlib.patches.Circle((main_drifter_path[-1,0], main_drifter_path[-1,1]), 6000, fill=False)
ax.add_patch(circ_end)



for path in paths:
    ax.plot(path[:,0], path[:,1], color='xkcd:light blue grey')
    
for path in paths:
    circ_end = matplotlib.patches.Circle((path[-1,0], path[-1,1]), 2000, fill=False, zorder=10)
    ax.add_patch(circ_end)
    
ax.plot(main_drifter_path[:,0], main_drifter_path[:,1], color='xkcd:dark grey blue')




circ_start = matplotlib.patches.Circle((extra_drifter_path[0,0], extra_drifter_path[0,1]), 6000, fill=False)
ax.add_patch(circ_start)

circ_end = matplotlib.patches.Circle((extra_drifter_path[-1,0], extra_drifter_path[-1,1]), 6000, fill=False)
ax.add_patch(circ_end)

for path in extra_paths:
    ax.plot(path[:,0], path[:,1], color='xkcd:foam green')
    
for path in extra_paths:
    circ_end = matplotlib.patches.Circle((path[-1,0], path[-1,1]), 2000, fill=False, zorder=10)
    ax.add_patch(circ_end)
    
ax.plot(extra_drifter_path[:,0], extra_drifter_path[:,1], color='xkcd:viridian')



print('hei')